# 1. Tạo tập test cho công cụ sửa lỗi chính tả

Tạo tập badword

In [ ]:
import pickle
def load(path):
  return pickle.load(open(path, 'rb'))

In [ ]:
# load dữ liệu lên
train_x = load('/content/drive/My Drive/Colab Notebooks/dump_hatespeech/train_x.dump')
train_y = load('/content/drive/My Drive/Colab Notebooks/dump_hatespeech/train_y.dump')

In [ ]:
# tách các bình luận thuộc các lớp ra
clean = []
hate_offensive = []
train_x = list(train_x)
for i in range(len(train_y)):
  if train_y[i,0] == 1:
    clean.append(train_x[i].replace('_', ' '))
  else:
    hate_offensive.append(train_x[i].replace('_', ' '))

In [ ]:
print("clean lenght                ", len(clean))
print("hate and offensive lenght   ", len(hate_offensive))

clean lenght                 14901
hate and offensive lenght    1375


In [ ]:
# chuyển các câu về dạng vector tần suất xuất hiện
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=8).fit(clean+hate_offensive*4)

clean_vec = vectorizer.transform(clean).toarray()
hate_offensive_vec = vectorizer.transform(hate_offensive).toarray()

In [ ]:
import numpy as np
# tính tổng tần suất của lớp clean và hate_offensive
clean_vec = np.sum(clean_vec, axis=0)
hate_offensive_vec = np.sum(hate_offensive_vec, axis=0)

In [ ]:
clean_vec

array([13,  9,  6, ..., 19, 10, 79])

In [ ]:
hate_offensive_vec

array([0, 0, 1, ..., 0, 3, 1])

In [ ]:
# lấy tần suất của lớp hate_ofensive trừ tần suất của lớp clean để tìm ra những từ độc hại (xuất hiện nhiều trong hate_offensive và ít xuất hiện trong clean)
sub = np.subtract(hate_offensive_vec, clean_vec)

In [ ]:
feature = vectorizer.get_feature_names()

In [ ]:
# Lấy ra những từ badword
badwords = []
feature = vectorizer.get_feature_names()
for i in range(len(sub)):
  if sub[i] > 2:
    badwords.append(feature[i])
print("badwords lenght", len(badwords))

In [ ]:
badwords

['bọn',
 'cc',
 'chó',
 'chịch',
 'chửi',
 'cút',
 'cứt',
 'dcm',
 'dm',
 'hãm',
 'lol',
 'lũ',
 'lồn',
 'mày',
 'mầy',
 'mồm',
 'ngu',
 'nhục',
 'sủa',
 'thằng',
 'tởm',
 'éo',
 'óc',
 'đcm',
 'đm',
 'đéo',
 'đĩ',
 'địt',
 'đụ']

Thống kê tần suất của các từ badword và sort chúng lại

In [ ]:
from collections import Counter

In [ ]:
count = Counter((' '.join(hate_offensive)).split())

In [ ]:
temp = dict()
for i in badwords:
  temp[i] = count[i]

In [ ]:
badwords = temp

In [ ]:
badwords = sorted(badwords.items(), key=lambda x: x[1], reverse=True)

In [ ]:
bws = badwords[:]

In [ ]:
s = ''
badwords = dict(badwords)
for i in badwords:
  s = s + i + ' ' + str(badwords[i]) + '\n'
s = s[:-1]

In [ ]:
# ghi tập badword xuống
open('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/badwords.txt', 'w').write(s)

421

Đọc tập dữ liệu thực tế và tìm các lỗi chính tả của các từ badword

In [ ]:
thin_data = open('/content/drive/My Drive/Colab Notebooks/BadWord/data_processed.txt').read().replace('\n', ' ')

In [ ]:
thin_count = Counter(thin_data.split())

In [ ]:
misspell = dict()
for i in thin_count:
  if thin_count[i] < 5:
    misspell[i] = thin_count[i]

In [ ]:
!pip install editdistance
from editdistance import eval as dist
misspell_badwords = []
for i in badwords:
  temp = dict()
  for j in misspell:
    if dist(i, j) <= 2:
      temp[j] = thin_count[j]
      temp = sorted(temp.items(), key=lambda x: x[1], reverse=True)
      temp = dict(temp)
  misspell_badwords.append(temp)

In [ ]:
s = ''
for i in range(len(badwords)):
  try:
    s = s + list(badwords)[i] + '\t' + ' '.join(misspell_badwords[i]) + '\n'
  except:
    s = s + list(badwords)[i] + '\n'
s = s[:-1]
open('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/auto_misspell_badwords.txt','w').write(s)

285974

Check lại tập badword sai chính tả đã chỉnh sửa thủ công

In [ ]:
mb_edited = open('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/misspell_badwords_edited.txt').read().split('\n')

In [ ]:
vocab = open('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/vocab.txt').read().split('\n')

In [ ]:
badwords = []
misspells = []
all_misspells = []
for i in mb_edited:
  bw, mss = i.strip().split('\t')
  if bw in dict(bws):
    temp = mss.split(' ')
    mss = []
    for j in temp:
      if j not in vocab:
        mss.append(j)
    badwords.append(bw)
    misspells.append(mss)
    all_misspells += mss

In [ ]:
len(badwords)

25

Tạo lỗi chính tả cho tập test

In [ ]:
import pandas as pd
orig_hate      = pd.read_csv('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/hate_and_offensive_orig_test/orig_hate.csv')
orig_offensive = pd.read_csv('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/hate_and_offensive_orig_test/orig_offensive.csv')

In [ ]:
def read_static(path):
  data = open(path).read().split('\n')
  result = dict()
  for i in data:
    try:
      word, freq = i.split()
      freq = int(freq)
      if freq >2:
        result[word] = freq
    except:
      None
  return result 

hate_word_static      = read_static('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/temp/word_statistics/hate_words.txt')
offensive_word_static = read_static('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/temp/word_statistics/offensive_words.txt')

In [ ]:
hate_offensive_cmts = list(orig_hate['comment']) + list(orig_offensive['comment'])
print('hate comment lenght       ', len(list(orig_hate['comment'])))
print('offensive comment lenght  ', len(list(orig_offensive['comment'])))

hate comment lenght        152
offensive comment lenght   204


In [ ]:
import string
import random
def change_a_word_dis1(word):
    Alphabet_List = list(string.ascii_lowercase)
    # Alphabet_List.append(' ')
    
    # 0 - add
    # 1 - delete
    # 2 - change    
    if len(word) <4:
      method = random.choice([0,2])
    else:
      method = random.randint(0, 2)
    
    if (method==0):
        pos = random.randint(0, len(word))
        word1 = word[0:pos]
        word2 = word[pos:len(word)]
        add = Alphabet_List[random.randint(0, len(Alphabet_List)-1)]
        return word1+add+word2
        
    elif (method==1):
        pos = random.randint(0, len(word)-1)
        word1 = word[0:pos]
        word2 = word[pos+1:len(word)]
        return word1+word2
        
    elif (method==2):
        pos = random.randint(0, len(word)-1)
        word1 = word[0:pos]
        word2 = word[pos+1:len(word)]
        change = word[pos]
        while (change==word[pos]):        
            change = Alphabet_List[random.randint(0, len(Alphabet_List)-1)]
        return word1+change+word2

def change_a_word(word):
  if len(word) < 5:
    return change_a_word_dis1(word)
  else:
    if random.choice([False, True]):
      return change_a_word_dis1(word)
    else:
      return change_a_word_dis1(change_a_word_dis1(word))

In [ ]:
import random
# tạo lỗi chính tả cho tập badword
correct_words = []
wrong_words   = []
wrong_hate_offensive_cmts = []
wrong_type = [] # 0: Lỗi thực tế, 1: Lỗi được thu thập từ bộ anh thìn, 2: Lỗi tự sinh
n = 0
for id, i in enumerate(hate_offensive_cmts):
  i = i.split()
  flag = False
  for id2, j in enumerate(i):
    # Nếu lỗi chính tả độc hại có sẵn trong trong bình luận
    if j in all_misspells:
      # tìm xem từ gốc của nó là từ nào
      for z in range(len(misspells)):
        if j in misspells[z]:
          correct_words.append(badwords[z])
          wrong_words.append(j)
          wrong_hate_offensive_cmts.append(' '.join(i))
          wrong_type.append(0)
          flag = True
          break
    if flag:
        break
  if flag:
    continue

  for id2, j in enumerate(i):
    # Nếu câu có chứa từ badword:
    if j in badwords:
      index = badwords.index(j)
      correct_words.append(badwords[index])
      wr_w = random.choice(misspells[index])
      wrong_words.append(wr_w)
      i[id2] = wr_w
      wrong_hate_offensive_cmts.append(' '.join(i))
      wrong_type.append(1)
      flag = True
      break
  if flag:
    continue

    # Nếu câu không chứa từ badword thì thêm tự động vào 
         
  if id <152:
    sents = dict()
    for j in i:
      try:
        sents[j] = hate_word_static[j]
      except:
        sents[j] = 0
    
    sents = dict(sorted(sents.items(), key=lambda x: x[1], reverse=True))
    for j in sents:
      try:
        wr_w = change_a_word(j)
        if wr_w in vocab and len(j) >= 2:
          continue
        correct_words.append(j)
        wrong_words.append(wr_w)
        i[i.index(j)] = wr_w
        wrong_hate_offensive_cmts.append(' '.join(i))
        wrong_type.append(2)
        n += 1
        break
      except:
        print(' '.join(i))
  else:
    sents = dict()
    for j in i:
      try:
        sents[j] = offensive_word_static[j]
      except:
        sents[j] = 0   

    sents = dict(sorted(sents.items(), key=lambda x: x[1], reverse=True))
    for j in sents:
      try:
        wr_w = change_a_word(j)
        if wr_w in vocab and len(j) >= 2:
          continue       
        correct_words.append(j)
        wrong_words.append(wr_w)
        i[i.index(j)] = wr_w
        wrong_hate_offensive_cmts.append(' '.join(i))
        wrong_type.append(2)
        n += 1
        break
      except:
        print(' '.join(i))

In [ ]:
print('correct word length            ', len(correct_words))
print('wrong word length              ', len(wrong_words))
print('hate and offensive lenght      ', len(hate_offensive_cmts))
print('wrong hate and offensive lenght', len(wrong_hate_offensive_cmts))

correct word length             356
wrong word length               356
hate and offensive lenght       356
wrong hate and offensive lenght 356


Ghi kết quả sau khi tạo lỗi chính tả xuống

In [ ]:
# ghi file sai chính tả xuống
# importing pandas as pd  
import pandas as pd  
   
# dictionary of lists  
temp = {'wrong_type': wrong_type,'correct_words': correct_words, 'wrong_words': wrong_words, 'hate_offensive_cmts': hate_offensive_cmts, 'wrong_hate_offensive_cmts':wrong_hate_offensive_cmts}  
     
df = pd.DataFrame(temp) 
  
# saving the dataframe 
df.to_csv('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/sai_chinh_ta_khong_dong_nghia.csv') 

# 2. Sửa lỗi chính tả cho tập dữ liệu hate và offensive

## Chỉ xét đúng về mặt hình thức

In [ ]:
import pandas as pd  
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/sai_chinh_ta_khong_dong_nghia.csv')
correct_words = data['correct_words']
wrong_words   = data['wrong_words']
hate_offensive_cmts = data['hate_offensive_cmts']
wrong_hate_offensive_cmts = data['wrong_hate_offensive_cmts']

correct_words = list(correct_words)
for i in range(len(correct_words)):
  correct_words[i] = correct_words[i].split('_')


In [ ]:
# correct_wrong_sents = open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/gg_output.txt').read().split('\n')

In [ ]:
# thống kê loại lỗi
print('Real error                      ', len(data[data['wrong_type'] == 0]))
print('Error create by Thin Dataset    ', len(data[data['wrong_type'] == 1]))
print('Generate error                  ', len(data[data['wrong_type'] == 2]))

Real error                       1
Error create by Thin Dataset     307
Generate error                   48


In [ ]:
import re 
!pip install pyvi
from pyvi import ViTokenizer
def processing_data(sent_list):
  sent_list = list(sent_list)
  temp = []
  for index, i in enumerate(sent_list):
    i = re.sub(r"(.)\1{2,}", r"\1\1", i)
    i = re.sub('\x01', '', i)
    i = re.sub('_|\s+', ' ', i)
    i = re.sub("""\!|\"|\#|\$|\%|\&|\||\'|\(|\)|\*|\+|\,|\-|\.|\/|\:|\;|\<|\=|\>|\?|\@|\[|\\|\]|\^|\_|\`|\{|\||\}|\~""", '', i)
    # i = i.replace('.','')
    # i = ViTokenizer.tokenize(i)
    sent_list[index] = i
  return sent_list

     |████████████████████████████████| 8.5MB 9.8MB/s 
     |████████████████████████████████| 747kB 43.4MB/s 


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/MaliciousSpellingCorrection - Copy')

Thử nghiệm với mô hình SKIP-GRAM, CBOW, FASTTEXT

In [ ]:
!pip install pyxDamerauLevenshtein
!pip install unidecode

     |████████████████████████████████| 61kB 3.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyxDamerauLevenshtein: filename=pyxDamerauLevenshtein-1.6.1-cp36-cp36m-linux_x86_64.whl size=85760 sha256=58b147bfcdc4b28dddff5f4a538fb930af29d27e68e0237170335ecad0ce7682
  Stored in directory: /root/.cache/pip/wheels/d2/05/c3/989929774aa6f86a368f18f132bcc79ed7cb1ea65763c1dc09
Successfully built pyxDamerauLevenshtein
     |████████████████████████████████| 245kB 9.3MB/s 


In [ ]:
method = ['skip-gram', 'cbow', 'fasttext_fb', 'fasttext_vne']
for m in method:
  from spelling_checking import spelling_checking
  small_corpus_dir = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/small_corpus1.txt'
  badwords_dir = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/hate_offensive_vocab.txt'
  spelling_checking = spelling_checking("/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/Model/"+m+"/", "vocab.txt", "vec.npy", small_corpus_dir, badwords_dir, CAND_LIMIT=8 )
  spelling_checking.readData()
  wrong_sents = processing_data(wrong_hate_offensive_cmts)
  # sửa cho tập train và lưu xuống
  correct_wrong_sents,_,_ = spelling_checking.generateAlgoCandCorrection(wrong_sents,3)

  temp = []
  for i in correct_wrong_sents:
    temp.append(' '.join(i))
  correct_wrong_sents = temp

  dung = 0
  sai = 0
  sot = 0
  s = ''
  for index, i in enumerate(correct_wrong_sents):
    i = i.split()
    # if correct_words[index] in i:
    if set(correct_words[index]) & set(i) != set():
      dung +=1
      s = (s  + 'ĐÚNG\n'
              + 'Từ sai:  ' + wrong_words[index] + '\t Từ gốc: ' +  '_'.join(correct_words[index]) + '\n'
              + 'Câu sửa: ' + correct_wrong_sents[index] + '\n'
              + 'Câu gốc: ' + hate_offensive_cmts[index] + '\n'
              + 'Câu sai: ' + wrong_hate_offensive_cmts[index] + '\n'
              + '----------------------------\n\n')
    else:
      sai += 1
      s = s + 'SAI'
      if wrong_words[index] in i:
        sot += 1 
        s = s + ' SÓT'
      s = (s  + '\n'
              + 'Từ sai:  ' + wrong_words[index] + '\t Từ gốc: ' +  '_'.join(correct_words[index]) + '\n'
              + 'Câu sửa: ' + correct_wrong_sents[index] + '\n'
              + 'Câu gốc: ' + hate_offensive_cmts[index] + '\n'
              + 'Câu sai: ' + wrong_hate_offensive_cmts[index] + '\n'
              + '----------------------------\n\n')
  # ghi file dữ liệu xuống
  open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/log_result/hate_offensive/' + m + '_kdn.txt', 'w').write(s)

  print("\n\n=================================")
  print("METHOD:  ", m.upper())
  print('SL đúng:', dung)
  print('SL sai :', sai)
  print('SL sót :', sot)
  print('Acc    :', dung*100/(dung+sai), '%')
  print('TL sót :', sot*100/(dung+sai), '%')
  print('TL phát hiện     :', 100 - sot*100/(dung+sai), '%')
  print('TL đúng/phát hiện:', dung*100/(dung+sai-sot), '%')

Done loading vocabulary.
Done loading vectors.
Corpus length          13509
Small corpus length    2069
Badwords length        1002
stage 1
done stage 1: raw check on edit distance
stage 2.
done stage 2: context-dependent check.


METHOD:   SKIP-GRAM
SL đúng: 267
SL sai : 89
SL sót : 9
Acc    : 75.0 %
TL sót : 2.5280898876404496 %
TL phát hiện     : 97.47191011235955 %
TL đúng/phát hiện: 76.94524495677233 %
Done loading vocabulary.
Done loading vectors.
Corpus length          13509
Small corpus length    2069
Badwords length        1002
stage 1
done stage 1: raw check on edit distance
stage 2.
done stage 2: context-dependent check.


METHOD:   CBOW
SL đúng: 260
SL sai : 96
SL sót : 13
Acc    : 73.03370786516854 %
TL sót : 3.651685393258427 %
TL phát hiện     : 96.34831460674157 %
TL đúng/phát hiện: 75.80174927113703 %
Done loading vocabulary.
Done loading vectors.
Corpus length          13023
Small corpus length    2069
Badwords length        1002
stage 1
done stage 1: raw check on edi

Thử nghiệm với mô hình symspell

In [ ]:
!pip install -U symspellpy
import pkg_resources
from symspellpy import SymSpell, Verbosity
# Khởi tạo
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
sym_spell.load_dictionary('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/SymSpell/monogram.txt', term_index=0, count_index=1)
sym_spell.load_bigram_dictionary('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/SymSpell/bigram.txt', term_index=0, count_index=2)
# sửa lỗi
correct_wrong_sents = []
for i in wrong_hate_offensive_cmts:
  # max edit distance per lookup (per single word, not per whole input string)
  suggestions = sym_spell.lookup_compound(i, max_edit_distance=2)
  correct_wrong_sents.append(suggestions[0].term)


In [ ]:
dung = 0
sai = 0
sot = 0
s = ''
# Đánh giá
for index, i in enumerate(correct_wrong_sents):
  i = i.split()
  # if correct_words[index] in i:
  if set(correct_words[index]) & set(i) != set():
    dung +=1
    s = (s  + 'ĐÚNG\n'
            + 'Từ sai:  ' + wrong_words[index] + '\t Từ gốc: ' +  '_'.join(correct_words[index]) + '\n'
            + 'Câu sửa: ' + correct_wrong_sents[index] + '\n'
            + 'Câu gốc: ' + hate_offensive_cmts[index] + '\n'
            + 'Câu sai: ' + wrong_hate_offensive_cmts[index] + '\n'
            + '----------------------------\n\n')
  else:
    sai += 1
    s = s + 'SAI'
    if wrong_words[index] in i:
      sot += 1 
      s = s + ' SÓT'
    s = (s  + '\n'
            + 'Từ sai:  ' + wrong_words[index] + '\t Từ gốc: ' +  '_'.join(correct_words[index]) + '\n'
            + 'Câu sửa: ' + correct_wrong_sents[index] + '\n'
            + 'Câu gốc: ' + hate_offensive_cmts[index] + '\n'
            + 'Câu sai: ' + wrong_hate_offensive_cmts[index] + '\n'
            + '----------------------------\n\n')
# ghi file dữ liệu xuống
open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/log_result/hate_offensive/symspell_kdn.txt', 'w').write(s)

print("\n\n=================================")
print("METHOD:  ", 'SYMSPELL')
print('SL đúng:', dung)
print('SL sai :', sai)
print('SL sót :', sot)
print('Acc    :', dung*100/(dung+sai), '%')
print('TL sót :', sot*100/(dung+sai), '%')
print('TL phát hiện     :', 100 - sot*100/(dung+sai), '%')
print('TL đúng/phát hiện:', dung*100/(dung+sai-sot), '%')



METHOD:   SYMSPELL
SL đúng: 253
SL sai : 103
SL sót : 16
Acc    : 71.06741573033707 %
TL sót : 4.49438202247191 %
TL phát hiện     : 95.50561797752809 %
TL đúng/phát hiện: 74.41176470588235 %


Thử nghiệm với GOOGLE SPELL CHECKER

In [ ]:
# load file google đã sửa lên
correct_wrong_sents = open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/gg_output.txt').read().split('\n')

In [ ]:
dung = 0
sai = 0
sot = 0
s = ''
# Đánh giá
for index, i in enumerate(correct_wrong_sents):
  i = i.split()
  # if correct_words[index] in i:
  if set(correct_words[index]) & set(i) != set():
    dung +=1
    s = (s  + 'ĐÚNG\n'
            + 'Từ sai:  ' + wrong_words[index] + '\t Từ gốc: ' +  '_'.join(correct_words[index]) + '\n'
            + 'Câu sửa: ' + correct_wrong_sents[index] + '\n'
            + 'Câu gốc: ' + hate_offensive_cmts[index] + '\n'
            + 'Câu sai: ' + wrong_hate_offensive_cmts[index] + '\n'
            + '----------------------------\n\n')
  else:
    sai += 1
    s = s + 'SAI'
    if wrong_words[index] in i:
      sot += 1 
      s = s + ' SÓT'
    s = (s  + '\n'
            + 'Từ sai:  ' + wrong_words[index] + '\t Từ gốc: ' +  '_'.join(correct_words[index]) + '\n'
            + 'Câu sửa: ' + correct_wrong_sents[index] + '\n'
            + 'Câu gốc: ' + hate_offensive_cmts[index] + '\n'
            + 'Câu sai: ' + wrong_hate_offensive_cmts[index] + '\n'
            + '----------------------------\n\n')
# ghi file dữ liệu xuống
open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/log_result/hate_offensive/google_kdn.txt', 'w').write(s)

print("\n\n=================================")
print("METHOD:  ", 'GOOGLE')
print('SL đúng:', dung)
print('SL sai :', sai)
print('SL sót :', sot)
print('Acc    :', dung*100/(dung+sai), '%')
print('TL sót :', sot*100/(dung+sai), '%')
print('TL phát hiện     :', 100 - sot*100/(dung+sai), '%')
print('TL đúng/phát hiện:', dung*100/(dung+sai-sot), '%')



METHOD:   GOOGLE
SL đúng: 143
SL sai : 213
SL sót : 168
Acc    : 40.168539325842694 %
TL sót : 47.19101123595506 %
TL phát hiện     : 52.80898876404494 %
TL đúng/phát hiện: 76.06382978723404 %


## Xét về mặt ngữ nghĩa

In [ ]:
import pandas as pd  
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/sai_chinh_ta_dong_nghia.csv')
correct_words = data['correct_words']
wrong_words   = data['wrong_words']
hate_offensive_cmts = data['hate_offensive_cmts']
wrong_hate_offensive_cmts = data['wrong_hate_offensive_cmts']

correct_words = list(correct_words)
for i in range(len(correct_words)):
  correct_words[i] = correct_words[i].split('_')


In [ ]:
# thống kê loại lỗi
print('Real error                      ', len(data[data['wrong_type'] == 0]))
print('Error create by Thin Dataset    ', len(data[data['wrong_type'] == 1]))
print('Generate error                  ', len(data[data['wrong_type'] == 2]))

Real error                       1
Error create by Thin Dataset     307
Generate error                   48


In [ ]:
import re 
!pip install pyvi
from pyvi import ViTokenizer
def processing_data(sent_list):
  sent_list = list(sent_list)
  temp = []
  for index, i in enumerate(sent_list):
    i = re.sub(r"(.)\1{2,}", r"\1\1", i)
    i = re.sub('\x01', '', i)
    i = re.sub('_|\s+', ' ', i)
    i = re.sub("""\!|\"|\#|\$|\%|\&|\||\'|\(|\)|\*|\+|\,|\-|\.|\/|\:|\;|\<|\=|\>|\?|\@|\[|\\|\]|\^|\_|\`|\{|\||\}|\~""", '', i)
    # i = i.replace('.','')
    # i = ViTokenizer.tokenize(i)
    sent_list[index] = i
  return sent_list

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/MaliciousSpellingCorrection - Copy')

Thử nghiệm với mô hình SKIP-GRAM, CBOW, FASTTEXT

In [ ]:
!pip install pyxDamerauLevenshtein
!pip install unidecode

In [ ]:
method = ['skip-gram', 'cbow', 'fasttext_fb', 'fasttext_vne']
for m in method:
  from spelling_checking import spelling_checking
  small_corpus_dir = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/small_corpus1.txt'
  badwords_dir = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/hate_offensive_vocab.txt'
  spelling_checking = spelling_checking("/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/Model/"+m+"/", "vocab.txt", "vec.npy", small_corpus_dir, badwords_dir, CAND_LIMIT=8 )
  spelling_checking.readData()
  wrong_sents = processing_data(wrong_hate_offensive_cmts)
  # sửa cho tập train và lưu xuống
  correct_wrong_sents,_,_ = spelling_checking.generateAlgoCandCorrection(wrong_sents,3)

  temp = []
  for i in correct_wrong_sents:
    temp.append(' '.join(i))
  correct_wrong_sents = temp

  dung = 0
  sai = 0
  sot = 0
  s = ''
  for index, i in enumerate(correct_wrong_sents):
    i = i.split()
    # if correct_words[index] in i:
    if set(correct_words[index]) & set(i) != set():
      dung +=1
      s = (s  + 'ĐÚNG\n'
              + 'Từ sai:  ' + wrong_words[index] + '\t Từ gốc: ' +  '_'.join(correct_words[index]) + '\n'
              + 'Câu sửa: ' + correct_wrong_sents[index] + '\n'
              + 'Câu gốc: ' + hate_offensive_cmts[index] + '\n'
              + 'Câu sai: ' + wrong_hate_offensive_cmts[index] + '\n'
              + '----------------------------\n\n')
    else:
      sai += 1
      s = s + 'SAI'
      if wrong_words[index] in i:
        sot += 1 
        s = s + ' SÓT'
      s = (s  + '\n'
              + 'Từ sai:  ' + wrong_words[index] + '\t Từ gốc: ' +  '_'.join(correct_words[index]) + '\n'
              + 'Câu sửa: ' + correct_wrong_sents[index] + '\n'
              + 'Câu gốc: ' + hate_offensive_cmts[index] + '\n'
              + 'Câu sai: ' + wrong_hate_offensive_cmts[index] + '\n'
              + '----------------------------\n\n')
  # ghi file dữ liệu xuống
  open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/log_result/hate_offensive/' + m + '_dn.txt', 'w').write(s)

  print("\n\n=================================")
  print("METHOD:  ", m.upper())
  print('SL đúng:', dung)
  print('SL sai :', sai)
  print('SL sót :', sot)
  print('Acc    :', dung*100/(dung+sai), '%')
  print('TL sót :', sot*100/(dung+sai), '%')
  print('TL phát hiện     :', 100 - sot*100/(dung+sai), '%')
  print('TL đúng/phát hiện:', dung*100/(dung+sai-sot), '%')

Done loading vocabulary.
Done loading vectors.
Corpus length          13509
Small corpus length    2069
Badwords length        1002
stage 1
done stage 1: raw check on edit distance
stage 2.
done stage 2: context-dependent check.


METHOD:   SKIP-GRAM
SL đúng: 291
SL sai : 65
SL sót : 9
Acc    : 81.74157303370787 %
TL sót : 2.5280898876404496 %
TL phát hiện     : 97.47191011235955 %
TL đúng/phát hiện: 83.86167146974063 %
Done loading vocabulary.
Done loading vectors.
Corpus length          13509
Small corpus length    2069
Badwords length        1002
stage 1
done stage 1: raw check on edit distance
stage 2.
done stage 2: context-dependent check.


METHOD:   CBOW
SL đúng: 276
SL sai : 80
SL sót : 13
Acc    : 77.52808988764045 %
TL sót : 3.651685393258427 %
TL phát hiện     : 96.34831460674157 %
TL đúng/phát hiện: 80.466472303207 %
Done loading vocabulary.
Done loading vectors.
Corpus length          13023
Small corpus length    2069
Badwords length        1002
stage 1
done stage 1: raw c

Thử nghiệm với mô hình symspell

In [ ]:
!pip install -U symspellpy
import pkg_resources
from symspellpy import SymSpell, Verbosity
# Khởi tạo
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
sym_spell.load_dictionary('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/SymSpell/monogram.txt', term_index=0, count_index=1)
sym_spell.load_bigram_dictionary('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/SymSpell/bigram.txt', term_index=0, count_index=2)
# sửa lỗi
correct_wrong_sents = []
for i in wrong_hate_offensive_cmts:
  # max edit distance per lookup (per single word, not per whole input string)
  suggestions = sym_spell.lookup_compound(i, max_edit_distance=2)
  correct_wrong_sents.append(suggestions[0].term)


     |████████████████████████████████| 2.6MB 11.7MB/s 


In [ ]:
dung = 0
sai = 0
sot = 0
s = ''
# Đánh giá
for index, i in enumerate(correct_wrong_sents):
  i = i.split()
  # if correct_words[index] in i:
  if set(correct_words[index]) & set(i) != set():
    dung +=1
    s = (s  + 'ĐÚNG\n'
            + 'Từ sai:  ' + wrong_words[index] + '\t Từ gốc: ' +  '_'.join(correct_words[index]) + '\n'
            + 'Câu sửa: ' + correct_wrong_sents[index] + '\n'
            + 'Câu gốc: ' + hate_offensive_cmts[index] + '\n'
            + 'Câu sai: ' + wrong_hate_offensive_cmts[index] + '\n'
            + '----------------------------\n\n')
  else:
    sai += 1
    s = s + 'SAI'
    if wrong_words[index] in i:
      sot += 1 
      s = s + ' SÓT'
    s = (s  + '\n'
            + 'Từ sai:  ' + wrong_words[index] + '\t Từ gốc: ' +  '_'.join(correct_words[index]) + '\n'
            + 'Câu sửa: ' + correct_wrong_sents[index] + '\n'
            + 'Câu gốc: ' + hate_offensive_cmts[index] + '\n'
            + 'Câu sai: ' + wrong_hate_offensive_cmts[index] + '\n'
            + '----------------------------\n\n')
# ghi file dữ liệu xuống
open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/log_result/hate_offensive/symspell_dn.txt', 'w').write(s)

print("\n\n=================================")
print("METHOD:  ", 'SYMSPELL')
print('SL đúng:', dung)
print('SL sai :', sai)
print('SL sót :', sot)
print('Acc    :', dung*100/(dung+sai), '%')
print('TL sót :', sot*100/(dung+sai), '%')
print('TL phát hiện     :', 100 - sot*100/(dung+sai), '%')
print('TL đúng/phát hiện:', dung*100/(dung+sai-sot), '%')



METHOD:   SYMSPELL
SL đúng: 269
SL sai : 87
SL sót : 7
Acc    : 75.56179775280899 %
TL sót : 1.9662921348314606 %
TL phát hiện     : 98.03370786516854 %
TL đúng/phát hiện: 77.07736389684814 %


Thử nghiệm với GOOGLE SPELL CHECKER

In [ ]:
# load file google đã sửa lên
correct_wrong_sents = open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/gg_output.txt').read().split('\n')

In [ ]:
dung = 0
sai = 0
sot = 0
s = ''
# Đánh giá
for index, i in enumerate(correct_wrong_sents):
  i = i.split()
  # if correct_words[index] in i:
  if set(correct_words[index]) & set(i) != set():
    dung +=1
    s = (s  + 'ĐÚNG\n'
            + 'Từ sai:  ' + wrong_words[index] + '\t Từ gốc: ' +  '_'.join(correct_words[index]) + '\n'
            + 'Câu sửa: ' + correct_wrong_sents[index] + '\n'
            + 'Câu gốc: ' + hate_offensive_cmts[index] + '\n'
            + 'Câu sai: ' + wrong_hate_offensive_cmts[index] + '\n'
            + '----------------------------\n\n')
  else:
    sai += 1
    s = s + 'SAI'
    if wrong_words[index] in i:
      sot += 1 
      s = s + ' SÓT'
    s = (s  + '\n'
            + 'Từ sai:  ' + wrong_words[index] + '\t Từ gốc: ' +  '_'.join(correct_words[index]) + '\n'
            + 'Câu sửa: ' + correct_wrong_sents[index] + '\n'
            + 'Câu gốc: ' + hate_offensive_cmts[index] + '\n'
            + 'Câu sai: ' + wrong_hate_offensive_cmts[index] + '\n'
            + '----------------------------\n\n')
# ghi file dữ liệu xuống
open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/log_result/hate_offensive/google_dn.txt', 'w').write(s)

print("\n\n=================================")
print("METHOD:  ", 'GOOGLE')
print('SL đúng:', dung)
print('SL sai :', sai)
print('SL sót :', sot)
print('Acc    :', dung*100/(dung+sai), '%')
print('TL sót :', sot*100/(dung+sai), '%')
print('TL phát hiện     :', 100 - sot*100/(dung+sai), '%')
print('TL đúng/phát hiện:', dung*100/(dung+sai-sot), '%')



METHOD:   GOOGLE
SL đúng: 152
SL sai : 204
SL sót : 167
Acc    : 42.69662921348315 %
TL sót : 46.91011235955056 %
TL phát hiện     : 53.08988764044944 %
TL đúng/phát hiện: 80.42328042328042 %


## abc

In [ ]:
from spelling_checking import spelling_checking

In [ ]:
small_corpus_dir = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/small_corpus1.txt'
badwords_dir = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/hate_offensive_vocab.txt'
# badwords_dir = '/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/temp/word_statistics/hate_offensive_words1.txt'
spelling_checking = spelling_checking("/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/Model/skip-gram/", "vocab.txt", "vec.npy", small_corpus_dir, badwords_dir, CAND_LIMIT=8 )

In [ ]:
spelling_checking.readData()

Done loading vocabulary.
Done loading vectors.
Corpus length          13509
Small corpus length    2069
Badwords length        1002


In [ ]:
# test thử 1 câu
error_sent_list = ["đến mấy con nguu bị lừa quay ra chê đàn ông"]
revised_sent_seq, algo_corrections, cand_corrections = spelling_checking.generateAlgoCandCorrection(error_sent_list, 2)
print("revised sent seq:")
print(revised_sent_seq)

stage 1
done stage 1: raw check on edit distance
stage 2.
done stage 2: context-dependent check.
revised sent seq:
[['đến', 'mấy', 'con', 'ngu', 'bị', 'lừa', 'quay', 'ra', 'chê', 'đàn', 'ông']]


In [ ]:
# def loaikitudau(word):
#   return word[0] + re.sub('w|s|f|r|x|j', '', word[1:])

In [ ]:
wrong_sents = processing_data(wrong_hate_offensive_cmts)

In [ ]:
# for i in range(len(wrong_sents)):
#   wrong_sents[i] = loaikitudau(wrong_sents[i])

In [ ]:
# sửa cho tập train và lưu xuống
correct_wrong_sents,_,_ = spelling_checking.generateAlgoCandCorrection(wrong_sents,3)

stage 1
done stage 1: raw check on edit distance
stage 2.
done stage 2: context-dependent check.


In [ ]:
temp = []
for i in correct_wrong_sents:
  temp.append(' '.join(i))
correct_wrong_sents = temp

In [ ]:
dung = 0
sai = 0
sot = 0
s = ''
for index, i in enumerate(correct_wrong_sents):
  i = i.split()
  # if correct_words[index] in i:
  if set(correct_words[index]) & set(i) != set():
    dung +=1
    s = (s  + 'ĐÚNG\n'
            + 'Từ sai:  ' + wrong_words[index] + '\t Từ gốc: ' +  '_'.join(correct_words[index]) + '\n'
            + 'Câu sửa: ' + correct_wrong_sents[index] + '\n'
            + 'Câu gốc: ' + hate_offensive_cmts[index] + '\n'
            + 'Câu sai: ' + wrong_hate_offensive_cmts[index] + '\n'
            + '----------------------------\n\n')
    # if (wrong_words[index] not in wrong_hate_offensive_cmts[index].split()[0]):
    #   print('Từ sai: ', wrong_words[index],'\t Từ gốc:', correct_words[index])
    #   print('Câu sửa:',correct_wrong_sents[index])
    #   print('Câu gốc:',hate_offensive_cmts[index])
    #   print('Câu sai:',wrong_hate_offensive_cmts[index])
    #   print('\n\n')
  else:
    sai += 1
    s = s + 'SAI'
    print("SAI")
    if wrong_words[index] in i:
      sot += 1 
      s = s + ' SÓT'
      print('SÓT')
    s = (s  + '\n'
            + 'Từ sai:  ' + wrong_words[index] + '\t Từ gốc: ' +  '_'.join(correct_words[index]) + '\n'
            + 'Câu sửa: ' + correct_wrong_sents[index] + '\n'
            + 'Câu gốc: ' + hate_offensive_cmts[index] + '\n'
            + 'Câu sai: ' + wrong_hate_offensive_cmts[index] + '\n'
            + '----------------------------\n\n')
      
      

    print('Từ sai:', wrong_words[index],'\t Từ gốc:', correct_words[index])
    print('Câu sửa:',correct_wrong_sents[index])
    print('Câu gốc:',hate_offensive_cmts[index])
    print('Câu sai:',wrong_hate_offensive_cmts[index])
    print('\n\n')

SAI
Từ sai: nguuu 	 Từ gốc: ['ngu']
Câu sửa: phân tích dài vl xúc tích mỗi tội nguu vl
Câu gốc: phân tích dài vl xúc tích mỗi tội ngu vl
Câu sai: phân tích dài vl xúc tích mỗi tội nguuu vl



SAI
Từ sai: lws 	 Từ gốc: ['lũ']
Câu sửa: nguyễn cao trọng nói này cái gì mấy đứa xàm có trên báo ngta nói là ngu dân
Câu gốc: nguyễn cao trọng nói này cái gì mấy đứa xàm có trên báo ngta nói lũ ngu dân
Câu sai: nguyễn cao trọng nói này cái gì mấy đứa xàm có trên báo ngta nói lws ngu dân



SAI
Từ sai: vsl 	 Từ gốc: ['vl']
Câu sửa: prince phùng bảo sao lấy xấu vkl
Câu gốc: prince phùng bảo sao lấy xấu vl
Câu sai: prince phùng bảo sao lấy xấu vsl



SAI
Từ sai: vsl 	 Từ gốc: ['vl']
Câu sửa: ad thâm dái vcl
Câu gốc: ad thâm dái vl
Câu sai: ad thâm dái vsl



SAI
Từ sai: đkó 	 Từ gốc: ['đó']
Câu sửa: lê thị ngọc tuyền đkm vailon
Câu gốc: lê thị ngọc tuyền đó condilon
Câu sai: lê thị ngọc tuyền đkó condilon



SAI
SÓT
Từ sai: cutt 	 Từ gốc: ['cút']
Câu sửa: xin nhắc lại nếu đã là bạn bè chơi với nhau 

In [ ]:
# open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/log_result/hate_offensive/fasttextvne_kdn.txt', 'w').write(s)

95399

In [ ]:
print('SL đúng:', dung)
print('SL sai :', sai)
print('SL sót :', sot)
print('Acc    :', dung*100/(dung+sai), '%')
print('TL sót :', sot*100/(dung+sai), '%')
print('TL phát hiện     :', 100 - sot*100/(dung+sai), '%')
print('TL đúng/phát hiện:', dung*100/(dung+sai-sot), '%')

SL đúng: 245
SL sai : 111
SL sót : 11
Acc    : 68.82022471910112 %
TL sót : 3.0898876404494384 %
TL phát hiện     : 96.91011235955057 %
TL đúng/phát hiện: 71.01449275362319 %


# 3. Sửa lỗi chính tả cho tập dữ liệu clean gốc

In [ ]:
import pickle
def load(path):
  return pickle.load(open(path, 'rb'))

train_x = load('/content/drive/My Drive/Colab Notebooks/dump_hatespeech/test_x.dump')
train_y = load('/content/drive/My Drive/Colab Notebooks/dump_hatespeech/test_y.dump')

clean = []
# hate_offensive = []
train_x = list(train_x)
for i in range(len(train_y)):
  if train_y[i,0] == 1:
    clean.append(train_x[i].replace('_', ' '))


In [ ]:
# open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/clean_test/clean_test.txt', 'w').write('\n'.join(clean))

355944

In [ ]:
clean_cmts = clean

In [ ]:
# open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/clean_test/gg_input.txt', 'w').write('\n'.join(wrong_clean_cmts))

In [ ]:
# !pip install -U symspellpy
# import pkg_resources
# from symspellpy import SymSpell, Verbosity

# sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
# # dictionary_path = pkg_resources.resource_filename(
# #     "symspellpy", "frequency_dictionary_en_82_765.txt")
# # bigram_path = pkg_resources.resource_filename(
# #     "symspellpy", "frequency_bigramdictionary_en_243_342.txt")
# # term_index is the column of the term and count_index is the
# # column of the term frequency
# sym_spell.load_dictionary('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/SymSpell/monogram.txt', term_index=0, count_index=1)
# sym_spell.load_bigram_dictionary('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/SymSpell/bigram.txt', term_index=0, count_index=2)

# # lookup suggestions for multi-word input strings (supports compound
# # splitting & merging)
# # input_term = ("lũ chó này tụi màyy biết tao là ai không")
# correct_wrong_sents = []
# for i in clean_cmts:
#   # max edit distance per lookup (per single word, not per whole input string)
#   suggestions = sym_spell.lookup_compound(i, max_edit_distance=2)
#   correct_wrong_sents.append(suggestions[0].term)
# # # display suggestion term, edit distance, and term frequency
# # for suggestion in suggestions:
# #     print(suggestion.term)

     |████████████████████████████████| 2.6MB 7.7MB/s 


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/MaliciousSpellingCorrection - Copy')

In [ ]:
!pip install pyxDamerauLevenshtein
!pip install unidecode

     |████████████████████████████████| 61kB 4.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyxDamerauLevenshtein: filename=pyxDamerauLevenshtein-1.6.1-cp36-cp36m-linux_x86_64.whl size=85765 sha256=6222f52c8f37993e26dccfa3ecb7dcee895ba9f90f0cf2f53c581c790089d30f
  Stored in directory: /root/.cache/pip/wheels/d2/05/c3/989929774aa6f86a368f18f132bcc79ed7cb1ea65763c1dc09
Successfully built pyxDamerauLevenshtein
     |████████████████████████████████| 245kB 12.0MB/s 


In [ ]:
from spelling_checking_nonBW import spelling_checking

In [ ]:
small_corpus_dir = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/small_corpus1.txt'
badwords_dir = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/hate_offensive_vocab.txt'
spelling_checking = spelling_checking("/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/Model/skip-gram/", "vocab.txt", "vec.npy", small_corpus_dir, badwords_dir, CAND_LIMIT=8 )

In [ ]:
spelling_checking.readData()

Done loading vocabulary.
Done loading vectors.
Corpus length          13509
Small corpus length    2069
Badwords length        1002


In [ ]:
# test thử 1 câu
error_sent_list = ["đến mấy con nguuu bị lừa quay ra chê đàn ông"]
revised_sent_seq, algo_corrections, cand_corrections = spelling_checking.generateAlgoCandCorrection(error_sent_list, 2)
print("revised sent seq:")
print(revised_sent_seq)

stage 1
done stage 1: raw check on edit distance
stage 2.
done stage 2: context-dependent check.
revised sent seq:
[['đến', 'mấy', 'con', 'ngu', 'bị', 'lừa', 'quay', 'ra', 'chê', 'đàn', 'ông']]


In [ ]:
import re 
!pip install pyvi
from pyvi import ViTokenizer
def processing_data(sent_list):
  sent_list = list(sent_list)
  temp = []
  for index, i in enumerate(sent_list):
    i = re.sub(r"(.)\1{2,}", r"\1\1", i)
    i = re.sub('\x01', '', i)
    i = re.sub('_|\s+', ' ', i)
    i = re.sub("""\!|\"|\#|\$|\%|\&|\||\'|\(|\)|\*|\+|\,|\-|\.|\/|\:|\;|\<|\=|\>|\?|\@|\[|\\|\]|\^|\_|\`|\{|\||\}|\~""", '', i)
    # i = i.replace('.','')
    # i = ViTokenizer.tokenize(i)
    sent_list[index] = i
  return sent_list

In [ ]:
# def loaikitudau(word):
#   return word[0] + re.sub('w|s|f|r|x|j', '', word[1:])

In [ ]:
wrong_sents = processing_data(clean_cmts)

In [ ]:
# for i in range(len(wrong_sents)):
#   wrong_sents[i] = loaikitudau(wrong_sents[i])

In [ ]:
# sửa cho tập train và lưu xuống
correct_wrong_sents,_,_ = spelling_checking.generateAlgoCandCorrection(wrong_sents,2)

stage 1
done stage 1: raw check on edit distance
stage 2.
done stage 2: context-dependent check.


In [ ]:
temp = []
for i in correct_wrong_sents:
  temp.append(' '.join(i))
correct_wrong_sents = temp

In [ ]:
badwords = open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/badwords.txt').read().split('\n')
for i in range(len(badwords)):
  badwords[i] = badwords[i].split()[0]

In [ ]:
# correct_wrong_sents = open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/clean_test/clean_test_gg.txt').read().split('\n')

In [ ]:
def badword_len(sent, badwords):
  sent = sent.split()
  num = 0
  bw = []
  for i in badwords:
    if i in sent:
      num += 1
      bw.append(i)
  return num, bw

In [ ]:
badword_sent_num = 0
mt_sent_num = 0
s = ''
for i in range(len(clean_cmts)):
  n1, bw1 = badword_len(clean_cmts[i], badwords)
  n2, bw2 = badword_len(correct_wrong_sents[i], badwords)
  if len(set(bw2) - set(bw1)) != 0:
    badword_sent_num +=1
    s = s + 'THÊM\n'
  else:
    s = s + 'KHÔNG\n'
  s = s + 'Thêm Badword: ' + str(set(bw2) - set(bw1)) + '\n'
  s = s + 'Từ bị sửa:    ' + str(set(clean_cmts[i].split()) -set(correct_wrong_sents[i].split())) + '\n'
  s = s + 'Câu gốc:      ' + str(clean_cmts[i]) + '\n'
  s = s + 'Câu sửa:      ' + str(correct_wrong_sents[i]) + '\n'
  s = s + '-----------------------\n\n'
  # print('Thêm Badword: ', set(bw2) - set(bw1))
  # print('Từ bị sửa:    ', set(clean_cmts[i].split()) -set(correct_wrong_sents[i].split()))
  # print('Câu gốc:      ', clean_cmts[i])
  # print('Câu sửa:      ', correct_wrong_sents[i])
  # print('-----------------------\n\n')

  if len(set(clean_cmts[i].split()) -set(correct_wrong_sents[i].split())) != 0:
    mt_sent_num += 1

In [ ]:
print('SL câu có thêm từ badword:     ', badword_sent_num)
print('TL câu có thêm từ badword:     ', badword_sent_num*100/len(clean_cmts), '%')
print('SL câu có từ thay đổi:         ', mt_sent_num)

SL câu có thêm từ badword:      142
TL câu có thêm từ badword:      3.824400754107191 %
SL câu có từ thay đổi:          1744


In [ ]:
open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/log_result/clean_goc/fasttext_vne.txt', 'w').write(s)

1085426

# 4. Tạo tập test trên tất cả các badword cho công cụ sửa lỗi chính tả

Tạo tập badword

In [ ]:
import pickle
def load(path):
  return pickle.load(open(path, 'rb'))

In [ ]:
train_x = load('/content/drive/My Drive/Colab Notebooks/dump_hatespeech/train_x.dump')
train_y = load('/content/drive/My Drive/Colab Notebooks/dump_hatespeech/train_y.dump')

In [ ]:
clean = []
hate_offensive = []
train_x = list(train_x)
for i in range(len(train_y)):
  if train_y[i,0] == 1:
    clean.append(train_x[i].replace('_', ' '))
  else:
    hate_offensive.append(train_x[i].replace('_', ' '))

In [ ]:
print("clean lenght                ", len(clean))
print("hate and offensive lenght   ", len(hate_offensive))

clean lenght                 14901
hate and offensive lenght    1375


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=8).fit(clean+hate_offensive*4)

In [ ]:
clean_vec = vectorizer.transform(clean).toarray()
hate_offensive_vec = vectorizer.transform(hate_offensive).toarray()

In [ ]:
import numpy as np
clean_vec = np.sum(clean_vec, axis=0)
hate_offensive_vec = np.sum(hate_offensive_vec, axis=0)

In [ ]:
clean_vec

array([13,  9,  6, ..., 19, 10, 79])

In [ ]:
hate_offensive_vec

array([0, 0, 1, ..., 0, 3, 1])

In [ ]:
sub = np.subtract(hate_offensive_vec, clean_vec)

In [ ]:
feature = vectorizer.get_feature_names()

In [ ]:
badwords = []
for i in range(len(sub)):
  if sub[i] > 2:
    badwords.append(feature[i])
print("badwords lenght", len(badwords))

badwords lenght 62


Thống kê tần suất của các từ badword và sort chúng lại

In [ ]:
from collections import Counter

In [ ]:
count = Counter((' '.join(hate_offensive)).split())

In [ ]:
temp = dict()
for i in badwords:
  temp[i] = count[i]

In [ ]:
badwords = temp

In [ ]:
badwords = sorted(badwords.items(), key=lambda x: x[1], reverse=True)

In [ ]:
s = ''
badwords = dict(badwords)
for i in badwords:
  s = s + i + ' ' + str(badwords[i]) + '\n'
s = s[:-1]

In [ ]:
open('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/badwords.txt', 'w').write(s)

421

Đọc tập dữ liệu thực tế và tìm các lỗi chính tả của các từ badword

In [ ]:
thin_data = open('/content/drive/My Drive/Colab Notebooks/BadWord/data_processed.txt').read().replace('\n', ' ')

In [ ]:
thin_count = Counter(thin_data.split())

In [ ]:
misspell = dict()
for i in thin_count:
  if thin_count[i] < 5:
    misspell[i] = thin_count[i]

In [ ]:
!pip install editdistance
from editdistance import eval as dist
misspell_badwords = []
for i in badwords:
  temp = dict()
  for j in misspell:
    if dist(i, j) <= 2:
      temp[j] = thin_count[j]
      temp = sorted(temp.items(), key=lambda x: x[1], reverse=True)
      temp = dict(temp)
  misspell_badwords.append(temp)

In [ ]:
s = ''
for i in range(len(badwords)):
  try:
    s = s + list(badwords)[i] + '\t' + ' '.join(misspell_badwords[i]) + '\n'
  except:
    s = s + list(badwords)[i] + '\n'
s = s[:-1]
open('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/auto_misspell_badwords.txt','w').write(s)

285974

Check lại tập badword sai chính tả đã chỉnh sửa thủ công

In [ ]:
mb_edited = open('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/misspell_badwords_edited.txt').read().split('\n')

In [ ]:
vocab = open('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/vocab.txt').read().split('\n')

In [ ]:
badwords = []
misspells = []
all_misspells = []
for i in mb_edited:
  bw, mss = i.strip().split('\t')
  temp = mss.split(' ')
  mss = []
  for j in temp:
    if j not in vocab:
      mss.append(j)
  badwords.append(bw)
  misspells.append(mss)
  all_misspells += mss

Tạo lỗi chính tả cho tập test

In [ ]:
import pandas as pd
orig_hate      = pd.read_csv('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/hate_and_offensive_orig_test/orig_hate.csv')
orig_offensive = pd.read_csv('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/hate_and_offensive_orig_test/orig_offensive.csv')

In [ ]:
def read_static(path):
  data = open(path).read().split('\n')
  result = dict()
  for i in data:
    try:
      word, freq = i.split()
      freq = int(freq)
      if freq >2:
        result[word] = freq
    except:
      None
  return result 

hate_word_static      = read_static('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/temp/word_statistics/hate_words.txt')
offensive_word_static = read_static('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/temp/word_statistics/offensive_words.txt')

In [ ]:
hate_offensive_cmts = list(orig_hate['comment']) + list(orig_offensive['comment'])
print('hate comment lenght       ', len(list(orig_hate['comment'])))
print('offensive comment lenght  ', len(list(orig_offensive['comment'])))

hate comment lenght        152
offensive comment lenght   204


In [ ]:
import string
import random
def change_a_word_dis1(word):
    Alphabet_List = list(string.ascii_lowercase)
    # Alphabet_List.append(' ')
    
    # 0 - add
    # 1 - delete
    # 2 - change    
    if len(word) <4:
      method = random.choice([0,2])
    else:
      method = random.randint(0, 2)
    
    if (method==0):
        pos = random.randint(0, len(word))
        word1 = word[0:pos]
        word2 = word[pos:len(word)]
        add = Alphabet_List[random.randint(0, len(Alphabet_List)-1)]
        return word1+add+word2
        
    elif (method==1):
        pos = random.randint(0, len(word)-1)
        word1 = word[0:pos]
        word2 = word[pos+1:len(word)]
        return word1+word2
        
    elif (method==2):
        pos = random.randint(0, len(word)-1)
        word1 = word[0:pos]
        word2 = word[pos+1:len(word)]
        change = word[pos]
        while (change==word[pos]):        
            change = Alphabet_List[random.randint(0, len(Alphabet_List)-1)]
        return word1+change+word2

def change_a_word(word):
  if len(word) < 5:
    return change_a_word_dis1(word)
  else:
    if random.choice([False, True]):
      return change_a_word_dis1(word)
    else:
      return change_a_word_dis1(change_a_word_dis1(word))

In [ ]:
import random
# tạo lỗi chính tả cho tập badword
correct_words = []
wrong_words   = []
wrong_hate_offensive_cmts = []
wrong_type = [] # 0: Lỗi thực tế, 1: Lỗi được thu thập từ bộ anh thìn, 2: Lỗi tự sinh
n = 0
for id, i in enumerate(hate_offensive_cmts):
  i = i.split()
  flag = False
  for id2, j in enumerate(i):
    # Nếu lỗi chính tả độc hại có sẵn trong trong bình luận
    if j in all_misspells:
      # tìm xem từ gốc của nó là từ nào
      for z in range(len(misspells)):
        if j in misspells[z]:
          correct_words.append(badwords[z])
          wrong_words.append(j)
          wrong_hate_offensive_cmts.append(' '.join(i))
          wrong_type.append(0)
          flag = True
          break
    if flag:
        break
  if flag:
    continue

  for id2, j in enumerate(i):
    # Nếu câu có chứa từ badword:
    if j in badwords:
      if flag == False:
        index = badwords.index(j)
        correct_words.append(badwords[index])
        wr_w = random.choice(misspells[index])
        wrong_words.append(wr_w)
        i[id2] = wr_w
        wrong_hate_offensive_cmts.append(' '.join(i))
        wrong_type.append(1)
        flag = True
      else:
        index = badwords.index(j)
        correct_words[-1] = correct_words[-1] + '|' + badwords[index]
        wr_w = random.choice(misspells[index])
        wrong_words[-1] = wrong_words[-1] + '|' + wr_w
        i[id2] = wr_w
        wrong_hate_offensive_cmts[-1] = ' '.join(i)
        # wrong_type.append(1)
        # flag = True
  if flag:
    continue

    # Nếu câu không chứa từ badword thì thêm tự động vào 
         
  if id <152:
    sents = dict()
    for j in i:
      try:
        sents[j] = hate_word_static[j]
      except:
        sents[j] = 0
    
    sents = dict(sorted(sents.items(), key=lambda x: x[1], reverse=True))
    for j in sents:
      try:
        wr_w = change_a_word(j)
        if wr_w in vocab and len(j) >= 2:
          continue
        correct_words.append(j)
        wrong_words.append(wr_w)
        i[i.index(j)] = wr_w
        wrong_hate_offensive_cmts.append(' '.join(i))
        wrong_type.append(2)
        n += 1
        break
      except:
        print(' '.join(i))
  else:
    sents = dict()
    for j in i:
      try:
        sents[j] = offensive_word_static[j]
      except:
        sents[j] = 0   

    sents = dict(sorted(sents.items(), key=lambda x: x[1], reverse=True))
    for j in sents:
      try:
        wr_w = change_a_word(j)
        if wr_w in vocab and len(j) >= 2:
          continue       
        correct_words.append(j)
        wrong_words.append(wr_w)
        i[i.index(j)] = wr_w
        wrong_hate_offensive_cmts.append(' '.join(i))
        wrong_type.append(2)
        n += 1
        break
      except:
        print(' '.join(i))


In [ ]:
print('correct word length            ', len(correct_words))
print('wrong word length              ', len(wrong_words))
print('hate and offensive lenght      ', len(hate_offensive_cmts))
print('wrong hate and offensive lenght', len(wrong_hate_offensive_cmts))

correct word length             356
wrong word length               356
hate and offensive lenght       356
wrong hate and offensive lenght 356


Ghi kết quả sau khi tạo lỗi chính tả xuống

In [ ]:
# ghi file sai chính tả xuống
# importing pandas as pd  
import pandas as pd  
   
# dictionary of lists  
temp = {'wrong_type': wrong_type,'correct_words': correct_words, 'wrong_words': wrong_words, 'hate_offensive_cmts': hate_offensive_cmts, 'wrong_hate_offensive_cmts':wrong_hate_offensive_cmts}  
     
df = pd.DataFrame(temp) 
  
# saving the dataframe 
df.to_csv('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/Tat_ca_cac_badword_loi/sai_chinh_ta_khong_dong_nghia.csv') 

## Sửa lỗi chính tả

In [ ]:
import pandas as pd  
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/Tat_ca_cac_badword_loi/sai_chinh_ta_khong_dong_nghia.csv')
correct_words = data['correct_words']
wrong_words   = data['wrong_words']
hate_offensive_cmts = data['hate_offensive_cmts']
wrong_hate_offensive_cmts = data['wrong_hate_offensive_cmts']

correct_words = list(correct_words)
for i in range(len(correct_words)):
  correct_words[i] = correct_words[i]


In [ ]:
# correct_wrong_sents = open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/gg_output.txt').read().split('\n')

In [ ]:
# thống kê loại lỗi
print('Real error                      ', len(data[data['wrong_type'] == 0]))
print('Error create by Thin Dataset    ', len(data[data['wrong_type'] == 1]))
print('Generate error                  ', len(data[data['wrong_type'] == 2]))


Real error                       1
Error create by Thin Dataset     307
Generate error                   48


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/MaliciousSpellingCorrection - Copy')

In [ ]:
!pip install pyxDamerauLevenshtein
!pip install unidecode

In [ ]:
method = ['skip-gram', 'cbow', 'fasttext_fb', 'fasttext_vne']
for m in method:
  from spelling_checking import spelling_checking
  small_corpus_dir = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/small_corpus1.txt'
  badwords_dir = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/hate_offensive_vocab.txt'
  spelling_checking = spelling_checking("/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/Model/"+m+"/", "vocab.txt", "vec.npy", small_corpus_dir, badwords_dir, CAND_LIMIT=8 )
  spelling_checking.readData()
  wrong_sents = processing_data(wrong_hate_offensive_cmts)
  # sửa cho tập train và lưu xuống
  correct_wrong_sents,_,_ = spelling_checking.generateAlgoCandCorrection(wrong_sents,4)

  temp = []
  for i in correct_wrong_sents:
    temp.append(' '.join(i))
  correct_wrong_sents = temp

  dung = 0
  sai = 0
  sot = 0
  s = ''
  for index, i in enumerate(correct_wrong_sents):
    i = i.split()
    # if correct_words[index] in i:
    cws = correct_words[index].split('|')
    wws = wrong_words[index].split('|')
    for j in range(len(cws)):
      if cws[j] in set(i):
        dung +=1
      else:
        sai += 1
        s = s + 'SAI'
        if wws[j] in i:
          sot += 1 
  print("\n\n=================================")
  print("METHOD:  ", m.upper())
  print('SL đúng:', dung)
  print('SL sai :', sai)
  print('SL sót :', sot)
  print('Acc    :', dung*100/(dung+sai), '%')
  print('TL sót :', sot*100/(dung+sai), '%')
  print('TL phát hiện     :', 100 - sot*100/(dung+sai), '%')
  print('TL đúng/phát hiện:', dung*100/(dung+sai-sot), '%')


Done loading vocabulary.
Done loading vectors.
Corpus length          13509
Small corpus length    2069
Badwords length        1002
stage 1
done stage 1: raw check on edit distance
stage 2.
done stage 2: context-dependent check.


METHOD:   SKIP-GRAM
SL đúng: 534
SL sai : 193
SL sót : 22
Acc    : 73.4525447042641 %
TL sót : 3.0261348005502064 %
TL phát hiện     : 96.97386519944979 %
TL đúng/phát hiện: 75.74468085106383 %
Done loading vocabulary.
Done loading vectors.
Corpus length          13509
Small corpus length    2069
Badwords length        1002
stage 1
done stage 1: raw check on edit distance
stage 2.
done stage 2: context-dependent check.


METHOD:   CBOW
SL đúng: 527
SL sai : 200
SL sót : 37
Acc    : 72.48968363136176 %
TL sót : 5.089408528198074 %
TL phát hiện     : 94.91059147180192 %
TL đúng/phát hiện: 76.3768115942029 %
Done loading vocabulary.
Done loading vectors.
Corpus length          13023
Small corpus length    2069
Badwords length        1002
stage 1
done stage 1: ra

In [ ]:
import re 
!pip install pyvi
from pyvi import ViTokenizer
def processing_data(sent_list):
  sent_list = list(sent_list)
  temp = []
  for index, i in enumerate(sent_list):
    i = re.sub(r"(.)\1{2,}", r"\1\1", i)
    i = re.sub('\x01', '', i)
    i = re.sub('_|\s+', ' ', i)
    i = re.sub("""\!|\"|\#|\$|\%|\&|\||\'|\(|\)|\*|\+|\,|\-|\.|\/|\:|\;|\<|\=|\>|\?|\@|\[|\\|\]|\^|\_|\`|\{|\||\}|\~""", '', i)
    # i = i.replace('.','')
    # i = ViTokenizer.tokenize(i)
    sent_list[index] = i
  return sent_list

THí nghiệm với symspell

In [ ]:
!pip install -U symspellpy
import pkg_resources
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

sym_spell.load_dictionary('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/SymSpell/monogram.txt', term_index=0, count_index=1)
sym_spell.load_bigram_dictionary('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/SymSpell/bigram.txt', term_index=0, count_index=2)

# lookup suggestions for multi-word input strings (supports compound
# splitting & merging)
# input_term = ("lũ chó này tụi màyy biết tao là ai không")
correct_wrong_sents = []
for i in wrong_hate_offensive_cmts:
  # max edit distance per lookup (per single word, not per whole input string)
  suggestions = sym_spell.lookup_compound(i, max_edit_distance=2)
  correct_wrong_sents.append(suggestions[0].term)
# # display suggestion term, edit distance, and term frequency
# for suggestion in suggestions:
#     print(suggestion.term)

     |████████████████████████████████| 2.6MB 7.7MB/s 


In [ ]:
dung = 0
sai = 0
sot = 0
s = ''
for index, i in enumerate(correct_wrong_sents):
  i = i.split()
  # if correct_words[index] in i:
  cws = correct_words[index].split('|')
  wws = wrong_words[index].split('|')
  for j in range(len(cws)):
    if cws[j] in set(i):
      dung +=1
      # s = (s  + 'ĐÚNG\n'
      #         + 'Từ sai:  ' + wrong_words[index] + '\t Từ gốc: ' +  '_'.join(correct_words[index]) + '\n'
      #         + 'Câu sửa: ' + correct_wrong_sents[index] + '\n'
      #         + 'Câu gốc: ' + hate_offensive_cmts[index] + '\n'
      #         + 'Câu sai: ' + wrong_hate_offensive_cmts[index] + '\n'
      #         + '----------------------------\n\n')
    else:
      sai += 1
      s = s + 'SAI'

      if wws[j] in i:
        sot += 1 
        s = s + ' SÓT'

      # s = (s  + '\n'
      #         + 'Từ sai:  ' + wrong_words[index] + '\t Từ gốc: ' +  '_'.join(correct_words[index]) + '\n'
      #         + 'Câu sửa: ' + correct_wrong_sents[index] + '\n'
      #         + 'Câu gốc: ' + hate_offensive_cmts[index] + '\n'
      #         + 'Câu sai: ' + wrong_hate_offensive_cmts[index] + '\n'
      #         + '----------------------------\n\n')


In [ ]:
open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/log_result/hate_offensive/fasttextvne_kdn.txt', 'w').write(s)

95322

In [ ]:
print('SL đúng:', dung)
print('SL sai :', sai)
print('SL sót :', sot)
print('Acc    :', dung*100/(dung+sai), '%')
print('TL sót :', sot*100/(dung+sai), '%')
print('TL phát hiện     :', 100 - sot*100/(dung+sai), '%')
print('TL đúng/phát hiện:', dung*100/(dung+sai-sot), '%')

SL đúng: 489
SL sai : 238
SL sót : 16
Acc    : 67.2627235213205 %
TL sót : 2.200825309491059 %
TL phát hiện     : 97.79917469050893 %
TL đúng/phát hiện: 68.77637130801688 %


Thí nghiệm với google

In [ ]:
open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/Tat_ca_cac_badword_loi/google.txt', 'w').write('\n'.join(wrong_hate_offensive_cmts))

21846

In [ ]:
correct_wrong_sents = open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/SuaLoiChinhTa/Tat_ca_cac_badword_loi/google_fix.txt').read().split('\n')

In [ ]:
for index, i in enumerate(correct_wrong_sents):
  i = i.split()
  # if correct_words[index] in i:
  cws = correct_words[index].split('|')
  wws = wrong_words[index].split('|')
  for j in range(len(cws)):
    if cws[j] in set(i):
      dung +=1
    else:
      sai += 1
      s = s + 'SAI'
      if wws[j] in i:
        sot += 1 

print('SL đúng:', dung)
print('SL sai :', sai)
print('SL sót :', sot)
print('Acc    :', dung*100/(dung+sai), '%')
print('TL sót :', sot*100/(dung+sai), '%')
print('TL phát hiện     :', 100 - sot*100/(dung+sai), '%')
print('TL đúng/phát hiện:', dung*100/(dung+sai-sot), '%')

SL đúng: 685
SL sai : 769
SL sót : 465
Acc    : 47.11141678129299 %
TL sót : 31.980742778541952 %
TL phát hiện     : 68.01925722145805 %
TL đúng/phát hiện: 69.26188068756319 %


Đánh giá recall của phần phát hiện lỗi

In [ ]:
method = ['skip-gram', 'cbow', 'fasttext_fb', 'fasttext_vne']
for m in method:
  from spelling_checking_recall import spelling_checking
  small_corpus_dir = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/small_corpus1.txt'
  badwords_dir = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/hate_offensive_vocab.txt'
  spelling_checking = spelling_checking("/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/Model/"+m+"/", "vocab.txt", "vec.npy", small_corpus_dir, badwords_dir, CAND_LIMIT=8 )
  spelling_checking.readData()
  # wrong_sents = processing_data(wrong_hate_offensive_cmts)

  ws = []
  w = []
  c = []
  for i in range(len(wrong_hate_offensive_cmts)):
    t1 = correct_words[i].split('|')
    t2 = wrong_words[i].split('|')
    for j in range(len(t1)):
      ws.append(wrong_hate_offensive_cmts[i])
      w.append(t2[j])
      c.append(t1[j])
  # sửa cho tập train và lưu xuống
  print("\n\n=================================")
  print("METHOD:  ", m.upper())
  print(spelling_checking.generateAlgoCandCorrection(ws,c, w))



Done loading vocabulary.
Done loading vectors.
Corpus length          13509
Small corpus length    2069
Badwords length        1002


METHOD:   SKIP-GRAM
stage 2.
Recall:  98.62448418156809
Pre:     82.28730822873082
None
Done loading vocabulary.
Done loading vectors.
Corpus length          13509
Small corpus length    2069
Badwords length        1002


METHOD:   CBOW
stage 2.
Recall:  98.62448418156809
Pre:     82.28730822873082
None
Done loading vocabulary.
Done loading vectors.
Corpus length          13023
Small corpus length    2069
Badwords length        1002


METHOD:   FASTTEXT_FB
stage 2.
Recall:  98.62448418156809
Pre:     82.28730822873082
None
Done loading vocabulary.
Done loading vectors.
Corpus length          13509
Small corpus length    2069
Badwords length        1002


METHOD:   FASTTEXT_VNE
stage 2.
Recall:  98.62448418156809
Pre:     82.28730822873082
None


In [ ]:
len(correct_words)

356